In [ ]:
import io
import numpy as np
import glob
import cv2

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from matplotlib import pyplot 
from PIL import Image
from skimage.metrics import structural_similarity as ssim

In [ ]:
# Parameters to change !

generatorPath = 'generator.h5'
imagesValPath = 'images/val128'

mseList = []
ssimList = []

In [ ]:
model = load_model(generatorPath)

In [ ]:
def load_image(pixels):
    pixels = img_to_array(pixels)
    pixels = (pixels - 127.5) / 127.5
    pixels = np.expand_dims(pixels, 0)
    
    return pixels

In [ ]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    return err

def compare_images(imageA, imageB, title):
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB)
    
    mseList.append(m)
    ssimList.append(s)
    
    '''
    # setup the figure
    fig = pyplot.figure(title)
    pyplot.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    pyplot.imshow(imageA, cmap = pyplot.cm.gray)
    pyplot.axis("off")
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    pyplot.imshow(imageB, cmap = pyplot.cm.gray)
    pyplot.axis("off")
    # show the images
    pyplot.show()
    '''

In [ ]:
def average(lst):
    return sum(lst) / len(lst)

In [ ]:
image_list = []
for filename in glob.glob(imagesValPath + '/*.png'):
    img = cv2.imread(filename)
    image_list.append(img)

sketchList = []
heightmapList = []
for image in image_list:
    width = img.shape[1]

    width_cutoff = width // 2
    s1 = image[:, :width_cutoff]
    s2 = image[:, width_cutoff:]
    sketchList.append(s1)
    heightmapList.append(s2)    

In [ ]:
for i in range(len(sketchList)):
    sketch = sketchList[i]
    destination = heightmapList[i]
    
    sketch = load_image(sketch)
    
    gen_image = model.predict(sketch)
    gen_image = (gen_image + 1) / 2.0    

    gen_image = (gen_image[0] * 255.0).astype(np.uint8)
    
    example_target = cv2.cvtColor(destination, cv2.COLOR_BGR2GRAY)
    gen_image = cv2.cvtColor(gen_image, cv2.COLOR_BGR2GRAY)
    
    compare_images(gen_image, example_target, "test")

In [ ]:
averageMse = average(mseList)
averageSsim = average(ssimList)

print(averageMse)
print(averageSsim)